In [ ]:
#default_exp core

# Core
> Core utility functions used in the library

In [ ]:
#export
from fastai2.vision.all import *

In [ ]:
#export
def _is_sequential(o): return o.__class__.__name__=='Sequential'
def _is_fn(o): return type(o)==types.FunctionType

In [ ]:
#export
def get_module(o,i):
  "Recursively get the module from list of indices"
  if is_listy(i):
    m = get_module(o,i[0])
    if len(i)==1: return m
    return get_module(m,i[1:])
  return o[i] if is_listy(o) else list(o.children())[i]

Most of the times, we want to extract nested module in architecture; while some modules support indexing (Sequential), some don't. This function enables you to access nested modules in a numpy-like indexing. When coupled with `arch_summary`, we can effortlessly explore the pytorch models.

In [ ]:
#export
def arch_summary(arch,idx=None,verbose:bool=False):
  r"""Short architecture summary, used for holistic understanding of model
  Args:
      arch: a function or model object
      idx (int,list): an integer or list of indices to reach desired module in the architecture
      verbose (bool): If True, will list the names of sub-modules
  """
  model = arch(False) if _is_fn(arch) else arch
  if idx is not None:
    model = get_module(model,idx)
  for i, l in enumerate(model.children()):
      n_layers = len(l if _is_sequential(l) else flatten_model(l))
      print(f'({i:<2}) {l.__class__.__name__:<17}: {n_layers:<4}layers')
      if verbose and l.has_children:
        layers = [x.__class__.__name__ for x in l.children()]
        for il in layers:
          print(" "*5,il)

## Examples

densenet121 from torchvision has first `_DenseLayer` at index [0,4,0], we can look at its brief summary as follows:

_(NB: densenet121 implementation is divided into two modules, body and head, directly skipping to body)_

In [ ]:
arch_summary(densenet121,0)

(0 ) Conv2d           : 1   layers
(1 ) BatchNorm2d      : 1   layers
(2 ) ReLU             : 1   layers
(3 ) MaxPool2d        : 1   layers
(4 ) _DenseBlock      : 36  layers
(5 ) _Transition      : 4   layers
(6 ) _DenseBlock      : 72  layers
(7 ) _Transition      : 4   layers
(8 ) _DenseBlock      : 144 layers
(9 ) _Transition      : 4   layers
(10) _DenseBlock      : 96  layers
(11) BatchNorm2d      : 1   layers


In [ ]:
arch_summary(densenet121,[0,4,0])

(0 ) BatchNorm2d      : 1   layers
(1 ) ReLU             : 1   layers
(2 ) Conv2d           : 1   layers
(3 ) BatchNorm2d      : 1   layers
(4 ) ReLU             : 1   layers
(5 ) Conv2d           : 1   layers


And if you really want to know go deeper, you may set `verbose=True` and `arch_summary` will go 2 depth down. For the simplicity, I'm keeping it to the depth of 2, since you can always have a detailed summary using `fastai2`'s patched summary method on module

In [ ]:
arch_summary(densenet121,0,verbose=True)

(0 ) Conv2d           : 1   layers
(1 ) BatchNorm2d      : 1   layers
(2 ) ReLU             : 1   layers
(3 ) MaxPool2d        : 1   layers
(4 ) _DenseBlock      : 36  layers
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
(5 ) _Transition      : 4   layers
      BatchNorm2d
      ReLU
      Conv2d
      AvgPool2d
(6 ) _DenseBlock      : 72  layers
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
(7 ) _Transition      : 4   layers
      BatchNorm2d
      ReLU
      Conv2d
      AvgPool2d
(8 ) _DenseBlock      : 144 layers
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _DenseLayer
      _Dens